In [11]:
import pandas as pd
import numpy as np
import nltk
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression # можно заменить на любимый классификатор
from sklearn.feature_extraction.text import CountVectorizer
from nltk import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import collocations 
from nltk.corpus import stopwords
from string import punctuation
from pymystem3 import Mystem
import re
from pymorphy2 import MorphAnalyzer
from razdel import tokenize
from natasha import Doc, MorphVocab, Segmenter, NewsEmbedding, NewsMorphTagger
from collections import Counter
import matplotlib.pyplot as plt

In [12]:
from nltk.corpus import stopwords
nltk.download('stopwords')
my_stopwords = stopwords.words('russian')
# noise = stopwords.words('russian') + list(punctuation)
noise = stopwords.words('russian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)

In [14]:
x_train, x_test, y_train, y_test = train_test_split(df.text, df.label)


In [15]:
def get_tokens_importance(maxdf, mindf):
    
    tfidf_vect = TfidfVectorizer(max_df=maxdf, # не берем слова что выше тресхолда
                                 min_df=mindf, # не берем что ниже тресхолда
#                                  max_features=1000, 
#                                  stop_words=noise,
                                 )
    
    bow = tfidf_vect.fit_transform(x_train)
    clf = LogisticRegression(random_state=42)
    clf.fit(bow, y_train)
    
    pred = clf.predict(tfidf_vect.transform(x_test))
    print(f'max_df= {maxdf} min_df= {mindf}')
    print(classification_report(pred, y_test))

In [16]:
# Берем все токены - f1-score получается низкая
get_tokens_importance(1, 1)

max_df= 1 min_df= 1
              precision    recall  f1-score   support

    negative       0.90      0.53      0.66     47540
    positive       0.22      0.68      0.33      9169

    accuracy                           0.55     56709
   macro avg       0.56      0.60      0.50     56709
weighted avg       0.79      0.55      0.61     56709



In [17]:
get_tokens_importance(0.7, 0.15)

max_df= 0.7 min_df= 0.15
              precision    recall  f1-score   support

    negative       0.34      0.61      0.44     15805
    positive       0.78      0.55      0.65     40904

    accuracy                           0.57     56709
   macro avg       0.56      0.58      0.54     56709
weighted avg       0.66      0.57      0.59     56709



In [18]:
get_tokens_importance(0.3, 1)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


max_df= 0.3 min_df= 1
              precision    recall  f1-score   support

    negative       0.73      0.77      0.75     26632
    positive       0.78      0.75      0.77     30077

    accuracy                           0.76     56709
   macro avg       0.76      0.76      0.76     56709
weighted avg       0.76      0.76      0.76     56709



На мой взгляд, менее встречающиеся токены важнее


Задание 2


In [21]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')
# corpus = [token for tweet in df.text for token in word_tokenize(tweet) if token not in punctuation]
corpus = [token for tweet in df.text for token in word_tokenize(tweet)]
print(len(corpus))
corpus[:10]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


4027060


['@', 'first_timee', 'хоть', 'я', 'и', 'школота', ',', 'но', 'поверь', ',']

In [22]:
freq_dict = Counter(corpus)
freq_dict_sorted= sorted(freq_dict.items(), key=lambda x: -x[1])
list(freq_dict_sorted)[:10]

[('(', 212404),
 (')', 194005),
 (',', 188295),
 (':', 177675),
 ('@', 149978),
 ('не', 69472),
 ('!', 66923),
 ('.', 57595),
 ('и', 55166),
 ('в', 52902)]